In [13]:
!pip install textblob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.3/624.3 kB 289.7 kB/s eta 0:00:00 0:00:01


In [9]:
import pandas as pd
import numpy as np
import ast
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
import scipy.sparse as sp
import os

In [10]:
pasta_treino = '../../data/files/treino'
pasta_itens = '../../data/itens/itens'

In [11]:
# Listar arquivos de cada pasta
arquivos_treino = [os.path.join(pasta_treino, f) for f in os.listdir(pasta_treino) if f.endswith('.csv')]
arquivos_itens = [os.path.join(pasta_itens, f) for f in os.listdir(pasta_itens) if f.endswith('.csv')]

In [ ]:
# Carregar arquivos de treinamento e itens em DataFrames únicos
training_data = pd.concat((pd.read_csv(f) for f in arquivos_treino), ignore_index=True)
# itens_data = pd.concat((pd.read_csv(f) for f in arquivos_itens), ignore_index=True)

In [ ]:
# Converter a coluna history para listas reais
def convert_to_list(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)  # Tenta avaliar como uma lista/dicionário
        except (ValueError, SyntaxError):  # Se falhar, converte a string em uma lista de strings separadas por vírgulas
            return x.split(',')  # ou outra lógica para transformar a string em uma lista
    return x

In [14]:
# Aplicando a conversão
training_data['history'] = training_data['history'].apply(convert_to_list)

In [15]:
# Explodir os históricos (cada item vira uma linha)

training_data = training_data.explode('history')

In [16]:
# Mesclar os dados de usuários com os atributos das notícias

training_data = training_data.merge(itens_data, left_on='history', right_on='page', how='left')

In [17]:
# Processar as datas corretamente
for col in ['issued', 'modified']:
    itens_data[col] = pd.to_datetime(itens_data[col], errors='coerce').dt.tz_localize(None)

In [18]:
# Criar features temporais
itens_data['news_age'] = (datetime.now() - itens_data['issued']).dt.days
itens_data['modification_frequency'] = (itens_data['modified'] - itens_data['issued']).dt.days

In [19]:
# Garantir que os valores de timestampHistory são strings e extrair números
training_data['timestampHistory'] = pd.to_numeric(
    training_data['timestampHistory'].astype(str).str.extract(r'(\d+)')[0],
    errors='coerce'
)

In [20]:
# Converter para datetime
training_data['timestampHistory'] = pd.to_datetime(training_data['timestampHistory'], unit='ms')

In [21]:
# Converter colunas numéricas corretamente
num_cols = ['timeOnPageHistory', 'numberOfClicksHistory', 'scrollPercentageHistory', 'pageVisitsCountHistory']
training_data[num_cols] = training_data[num_cols].apply(pd.to_numeric, errors='coerce')

In [24]:
# Criar features de usuário
training_data['total_news_read'] = training_data.groupby('userId')['history'].transform('count')
training_data['mean_time_on_page'] = training_data.groupby('userId')['timeOnPageHistory'].transform('mean')
training_data['var_time_on_page'] = training_data.groupby('userId')['timeOnPageHistory'].transform('var')
training_data['recency'] = (datetime.now() - training_data['timestampHistory']).dt.days

In [25]:
# Converte as colunas para datetime sem fuso horário (tz-naive)
itens_data['issued'] = pd.to_datetime(itens_data['issued']).dt.tz_localize(None)
itens_data['modified'] = pd.to_datetime(itens_data['modified']).dt.tz_localize(None)

# Agora, cria as features
itens_data['news_age'] = (datetime.now() - itens_data['issued']).dt.days
itens_data['modification_frequency'] = (itens_data['modified'] - itens_data['issued']).dt.days

In [26]:
# Baixar stopwords apenas uma vez
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nikolly/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
from nltk.corpus import stopwords

In [28]:
# Converter para lista
stop_words = list(stopwords.words('portuguese'))

In [29]:
# Criar o vectorizer com as stopwords em português
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words)
tfidf_matrix = tfidf_vectorizer.fit_transform(itens_data['body'].fillna(''))

In [30]:
# Selecionar features de usuário
user_feature_cols = ['total_news_read', 'mean_time_on_page', 'var_time_on_page', 'recency']
user_features = training_data[['userId'] + user_feature_cols].drop_duplicates()

In [31]:
# Criar matriz TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words)
tfidf_matrix = tfidf_vectorizer.fit_transform(itens_data['body'].fillna(''))

In [32]:
# Normalizar features numéricas
feature_cols = ['news_age', 'modification_frequency']
itens_data_features = itens_data[feature_cols].fillna(0)
scaler = MinMaxScaler()
itens_data_features_scaled = scaler.fit_transform(itens_data_features)

In [33]:
# Normalizar features de usuário
scaler_user = MinMaxScaler()
user_features_scaled = scaler_user.fit_transform(user_features[user_feature_cols])

/home/nikolly/fiap/fiap-tech-challenge-5/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/nikolly/fiap/fiap-tech-challenge-5/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


In [ ]:
# Transformar categorias em vetores numéricos
encoder = OneHotEncoder(handle_unknown='ignore')
categorias_encoded = encoder.fit_transform(itens_data[['category']])

In [ ]:
# Combinar TF-IDF com features numéricas, categorias e usuário
tfidf_plus_features_users = sp.hstack((tfidf_matrix, itens_data_features_scaled, categorias_encoded))


In [ ]:

# Calcular similaridade
similaridade_completa = cosine_similarity(tfidf_plus_features_users)